In [ ]:
!pip install PyMuPDF
!pip install google-generativeai

In [ ]:
import sys
!{sys.executable} -m pip install PyMuPDF google-generativeai

In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox
import fitz
import google.generativeai as genai

## Gemeni Config.

In [24]:
genai.configure(api_key="api-key")

## Extract PDF

In [10]:
def extract_text(path):
    text = ""
    with fitz.open(path) as doc:
        for page in doc:
            text += page.get_text()
    return text

## Generate Flashcards using Gemeni

In [11]:
def generate_flashcards(pdf_text):
    prompt = f"""
        Based on the following text, create 5 multiple choice questions with 4 options each.
        Format:

        Q: <question>
        A:
        - A. option
        - B. option
        - C. option
        - D. option
        Answer: <letter>

        TEXT: 
        \"\"\"
        {pdf_text[:3000]}
        \"\"\"
        Only return the question in the format described.
    """
    try:
        model = genai.GenerativeModel('gemini-2.5-flash')
        response = model.generate_content(prompt)
        print("Gemeni Output: \n", response.text)
        return parse_flashcards(response.text)
    except Exception as e:
        print("Gemini Error:", e)
        return[]

## Parse Gemeni Responses into Flashcards

In [12]:
def parse_flashcards(text):
    flashcards = []
    chunks = text.strip().split("Q:")
    for chunk in chunks[1:]:
        try:
            lines = [line.strip() for line in chunk.strip().split("\n") if line.strip()]
            question = lines[0]
            options = []
            
            for line in lines[1:]:
                # Look for lines starting with A., B., C., or D.
                if line.startswith(("- A.", "- B.", "- C.", "- D.")):
                    parts = line.split(".", 1)
                    if len(parts) == 2:
                        options.append(parts[1].strip())
                if len(options) == 4:
                    break
            
            # Find the answer line
            answer_line = next((line for line in lines if "Answer:" in line), None)
            if answer_line:
                # Extract just the letter after "Answer:"
                answer_part = answer_line.split(":")[-1].strip().upper()
                # Get first character in case there's extra text
                correct_letter = answer_part[0] if answer_part else None
                
                if correct_letter in ["A", "B", "C", "D"]:
                    correct_index = ["A", "B", "C", "D"].index(correct_letter)
                    if len(options) == 4:
                        flashcards.append({
                            "question": question,
                            "options": options,
                            "correct": correct_index
                        })
        except Exception as e:
            print(f"Error parsing chunk: {e}")
            continue
    
    return flashcards

## GUI

In [19]:
class FlashcardQuizApp:
    def __init__(self, master):
        self.master = master
        self.master.title("AI FlashCards")
        self.master.geometry("1000x700")
    
        self.flashcards = []
        self.current_index = 0
        self.score = 0
        self.selected_option = tk.IntVar()
        self.user_answers = []  # Moved here for better organization
    
        self.title_label = tk.Label(master, text="Load PDF to Begin", font=("Arial", 14, "bold"))
        self.title_label.pack(pady=10)
    
        self.load_btn = tk.Button(master, text="Load PDF", command=self.load_pdf)
        self.load_btn.pack(pady=5)
        
        self.start_btn = tk.Button(master, text="Start Quiz", state="disabled", command=self.start_quiz)
        self.start_btn.pack(pady=5)
    
        self.question_label = tk.Label(master, text="", wraplength=500, justify="left", font=("Arial", 12))
        self.option_buttons = [
            tk.Radiobutton(master, text="", variable=self.selected_option, value=i, font=("Arial", 12))
            for i in range(4)
        ]
    
        self.next_btn = tk.Button(master, text="Next", command=self.next_question)
        self.quit_btn = tk.Button(master, text="Cancel", command=master.destroy)
    
    def load_pdf(self):
        path = filedialog.askopenfilename(filetypes=[("PDF files", "*.pdf")])
        if path:
            self.title_label.config(text="Reading PDF ...")
            self.master.update()
            pdf_text = extract_text(path)
            self.title_label.config(text="Generating Flashcards ...")
            self.master.update()
            self.flashcards = generate_flashcards(pdf_text)
            if self.flashcards:
                messagebox.showinfo("Done", f"Loaded {len(self.flashcards)} flashcards!")
                self.start_btn.config(state="normal")
                self.title_label.config(text="Ready! Click 'Start Quiz'")
            else:
                messagebox.showerror("Error", "Failed to generate flashcards.")
    
    def start_quiz(self):
        self.score = 0
        self.current_index = 0
        self.user_answers = []  # Reset answers
        self.title_label.pack_forget()
        self.load_btn.pack_forget()
        self.start_btn.pack_forget()
        self.display_question()
    
    def display_question(self):
        if self.current_index >= len(self.flashcards):
            self.end_quiz()
            return
    
        flash = self.flashcards[self.current_index]
        self.selected_option.set(-1)
        self.question_label.config(text=f"Q{self.current_index + 1}: {flash['question']}")
        self.question_label.pack(pady=10)
    
        for i, opt in enumerate(flash["options"]):
            self.option_buttons[i].config(text=opt)
            self.option_buttons[i].pack(anchor="w", padx=20)
    
        self.next_btn.pack(pady=10)
        self.quit_btn.pack()
    
    def next_question(self):
        selected = self.selected_option.get()
        if selected == -1:
            messagebox.showwarning("Warning", "Please select an option")
            return
        
        self.user_answers.append(selected)
        
        if selected == self.flashcards[self.current_index]["correct"]:
            self.score += 1
        
        self.current_index += 1
        self.display_question()
    
    def end_quiz(self):
        for btn in self.option_buttons:
            btn.pack_forget()
        
        results_text = f"Quiz Complete!\nYour Score: {self.score} / {len(self.flashcards)}\n\n"
        results_text += "Review:\n"
        results_text += "-" * 50 + "\n"
        
        for i, flash in enumerate(self.flashcards):
            user_ans = self.user_answers[i]
            correct_ans = flash["correct"]
            status = "✓ CORRECT" if user_ans == correct_ans else "✗ INCORRECT"
            
            results_text += f"\nQ{i+1}: {status}\n"
            results_text += f"Question: {flash['question']}\n\n"
            
            if user_ans != correct_ans:
                results_text += f"Your answer: {['A', 'B', 'C', 'D'][user_ans]}. {flash['options'][user_ans]}\n"
                results_text += f"Correct answer: {['A', 'B', 'C', 'D'][correct_ans]}. {flash['options'][correct_ans]}\n"
            else:
                results_text += f"You correctly chose: {['A', 'B', 'C', 'D'][correct_ans]}. {flash['options'][correct_ans]}\n"
            
            results_text += "-" * 50 + "\n"
        
        self.question_label.config(text=results_text)
        self.next_btn.pack_forget()
        self.quit_btn.config(text="Exit")

## Launch GUI

In [23]:
if __name__ == "__main__":
    root = tk.Tk()
    app = FlashcardQuizApp(root)
    root.mainloop()

Gemeni Output: 
 Q: What is the primary purpose of the "Computer Science Fundamentals Curriculum Guide"?
A:
- A. To provide a historical overview of computer science education.
- B. To help users navigate lessons in Courses A-F and offer teaching tips.
- C. To market new computer science software to educators.
- D. To detail the history of Code.org's curriculum development.
Answer: B

Q: The "CS Fundamentals Curriculum Guide" is specifically designed to help users with which range of courses?
A:
- A. Courses 1-6
- B. Courses K-12
- C. Courses A-F
- D. Advanced Placement (AP) Computer Science courses
Answer: C

Q: How are the curriculum resources and tutorials described in the guide licensed?
A:
- A. They are proprietary and require a paid subscription.
- B. They are available under a Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License.
- C. They are open source for technology, but curriculum requires a separate license.
- D. They are free for personal use bu